# Load ImageNet images
Imagenet images are subject to copyright and some could image urls may be outdated and invalid. So there's a chance that the image could not be obtained. 

In [130]:
import numpy as np
import requests
import cv2
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlretrieve
import json
import os.path

bold_dir = "/Users/jz/Desktop/BOLD5000/"

In [127]:
def geturl(imageName):
    wnid = imageName[:imageName.find('_')]
    image_urls = "http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=" + wnid
    url_mappings = "http://www.image-net.org/api/text/imagenet.synset.geturls.getmapping?wnid=" + wnid
    '''
    page = requests.get(image_urls)
    soup = BeautifulSoup(page.content, 'html.parser')
    str_soup = str(soup)
    urls = str_soup.split('\r\n')
    #print(urls[:100])
    '''
    imgName = imageName[:imageName.find('.')]
    mapping_page = requests.get(url_mappings)
    mapping_soup = BeautifulSoup(mapping_page.content, 'html.parser')
    mapping_str = str(mapping_soup)
    
    i = mapping_str.find(imgName)
    if i == -1:
        return False
    start_index = mapping_str.find(' ', i) + 1
    end_index = mapping_str.find('\r\n', start_index)
    url = mapping_str[start_index:end_index]
    
    # Parse unicode string to python-safe url
    url = urllib.parse.quote(url, safe=':/')
    return url
    
#     mappings = mapping_str.split('\r\n')
#     for mapping in mappings:
#         name = mapping[:mapping.find(' ')]
#         #print(imageName[:imageName.find('.')])
#         if name == imgName:
#             url = mapping[mapping.find(' ')+1:]
#             return url
#     return False

In [128]:
geturl('n07695742_5848.JPEG')

'http://farm4.static.flickr.com/3561/3330359073_e07d7c4f90.jpg'

## 1×× Informational
100 Continue
101 Switching Protocols
102 Processing
## 2×× Success
200 OK
201 Created
202 Accepted
203 Non-authoritative Information
204 No Content
205 Reset Content
206 Partial Content
207 Multi-Status
208 Already Reported
226 IM Used
## 3×× Redirection
300 Multiple Choices
301 Moved Permanently
302 Found
303 See Other
304 Not Modified
305 Use Proxy
307 Temporary Redirect
308 Permanent Redirect
## 4×× Client Error
400 Bad Request
401 Unauthorized
402 Payment Required
403 Forbidden
404 Not Found
405 Method Not Allowed
406 Not Acceptable
407 Proxy Authentication Required
408 Request Timeout
409 Conflict
410 Gone
411 Length Required
412 Precondition Failed
413 Payload Too Large
414 Request-URI Too Long
415 Unsupported Media Type
416 Requested Range Not Satisfiable
417 Expectation Failed
418 I'm a teapot
421 Misdirected Request
422 Unprocessable Entity
423 Locked
424 Failed Dependency
426 Upgrade Required
428 Precondition Required
429 Too Many Requests
431 Request Header Fields Too Large
444 Connection Closed Without Response
451 Unavailable For Legal Reasons
499 Client Closed Request
## 5×× Server Error
500 Internal Server Error
501 Not Implemented
502 Bad Gateway
503 Service Unavailable
504 Gateway Timeout
505 HTTP Version Not Supported
506 Variant Also Negotiates
507 Insufficient Storage
508 Loop Detected
510 Not Extended
511 Network Authentication Required
599 Network Connect Timeout Error

In [133]:

            
for session in range(1,4):
    # Open the stimlist for reading
    with open('{}BOLD5000_ROIs/ROIs/stim_lists/CSI0{}_stim_lists.txt'.format(bold_dir, session+1), 'r') as f:
        stimlist = f.read()
    
    img_list = stimlist.split('\n')
    os.path.isfile()
    data = {}
    for img_name in img_list:
        # For some reason the last string is empty and incurs error
        if (img_name[0] == 'n') and (len(img_name)>0):
            if os.path.isfile("{}BOLD5000_ROIs/ROIs/CSI{}/images/{}".format(bold_dir, session+1, img_name)):
                print('{} already exists!'.format(img_name))
                continue
            elif 
            img_url = geturl(img_name)
            # Could not find image url in mapping
            if img_url == False:
                print("Img url invalid")
                data[img_name] = ""
                continue
            # Get status code 
            try:
                header = requests.head(img_url, allow_redirects=True)
                code = header.status_code
            except requests.ConnectionError:
                print("failed to connect")
                data[img_name] = ""

            if (int(code/100)==2) or (int(code/100)==3) or (int(code%100)==1):
                data[img_name] = img_url
                try:
                    urlretrieve(img_url, "{}BOLD5000_ROIs/ROIs/CSI{}/images/{}".format(bold_dir, session+1, img_name))
                except Exception as e:
                    print('------------------')
                    print(e)
                    print(img_url)
                    print('Status code {}'.format(code))
                    data[img_name] = ""
            else:
                print('------------------')
                print(img_url)
                print('Status code {}'.format(code))
                data[img_name] = ""
                    
    with open('/Users/jz/Desktop/BOLD5000/BOLD5000_ROIs/ROIs/CSI{}.json'.format(session+1), 'w') as f:
        json.dump(data, f)

------------------
http://www.ae.uiuc.edu/~arakoni/Whitaker%2520point/valley%2520below.jpg
Status code 404
n01494475_3853.JPEG already exists!
------------------
http://farm3.static.flickr.com/2168/1508447155_c069fbbdfb.jpg
Status code 404
------------------
http://static.flickr.com/2040/1518691594_94b62809e5.jpg
Status code 404
------------------
http://farm4.static.flickr.com/3599/3309395217_b57e605b42.jpg
Status code 404
------------------
http://www.monkeyingabout.com/images/hatsandcaps/cowboy_hat_21/cowboy_hat_212.jpg
Status code 404
n02111129_1827.JPEG already exists!
n04192698_5041.JPEG already exists!
n01983481_4719.JPEG already exists!
failed to connect
------------------
http://www.furniture800.com/images/Day-Bed-K1720.jpg
Status code 404
------------------
http://farm1.static.flickr.com/208/467479819_d385c89803.jpg
Status code 404
------------------
http://www.falegnameriagrossi.it/public/usato/armadio%2520ciliegio-interno_0.jpg
Status code 404
------------------
http://farm

------------------
http://www.johnnyjet.com/images/PicForNewsLetterFijiSnorkelNov2004JJSnorkel.jpg
Status code 403
------------------
http://enewbold.com/images/bees_06032007/PICT0001a.jpg
Status code 404
------------------
http://www.naturfotografen-forum.de/data/media/9/FBDB3726_RJ::Achim_Winkler_tiger_raubkatzen_zoo_duisburger.jpg
Status code 403
------------------
http://www.dougieonline.com/pictures/2003/kyoto2003/day1/rickshaw1.jpg
Status code 404
------------------
http://www.parcodessi.it/parcodessi/export/sites/default/www/Schedario/Immagini/chiesaConventoSantaLucia01g.jpg
Status code 404
Img url invalid
------------------
http://www.patrickmavros.com/products/large_images/1082722536.jpg
Status code 404
n02088632_2841.JPEG already exists!
------------------
http://farm3.static.flickr.com/2067/2244898970_5274254cb4.jpg
Status code 404
n02730930_15174.JPEG already exists!
n07718472_4516.JPEG already exists!
n02108089_344.JPEG already exists!
failed to connect
------------------


------------------
http://i4.photobucket.com/albums/y121/pactio_kiss/Image000.jpg
Status code 404
n02127052_7986.JPEG already exists!
n01817953_6235.JPEG already exists!
------------------
http://www.cwca.org.cn/Article/UploadFiles/200604/20060402151132186.jpg
Status code 404
n04548362_5813.JPEG already exists!
n09332890_13407.JPEG already exists!
failed to connect
------------------
http://vermilionkennels.com/wp-content/uploads/2006/02/Jenny%2520Peterson%2520002.bmp
Status code 404
n02113023_4356.JPEG already exists!
n04485082_10204.JPEG already exists!
------------------
http://farm2.static.flickr.com/1277/642721969_4bb5ebdcfe.jpg
Status code 404
------------------
http://www.texasoesrescue.org/Max02.jpg
Status code 403
------------------
http://lh5.ggpht.com/_q7J-xE0SreI/RvBEYIfkzPI/AAAAAAAABPg/u7qS0dmCnEM/100_2998.jpg
Status code 404
n03891332_5084.JPEG already exists!
Img url invalid
n03047690_43845.JPEG already exists!
------------------
http://www.ssaddle.com/images%2520boots/s

ConnectionError: HTTPConnectionPool(host='www.image-net.org', port=80): Max retries exceeded with url: /api/text/imagenet.synset.geturls.getmapping?wnid=n03977966 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11c0f3d90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [115]:
with open('/Users/jz/Desktop/BOLD5000/BOLD5000_ROIs/ROIs/CSI{}.json'.format(0+1), 'w') as f:
    json.dump(data, f)

In [122]:
fails = 0
successes = 0
for (key, value) in data.items():
    if value == '':
        fails += 1
    else:
        successes += 1
print(successes, "successes")
print(fails, "fails")

1060 successes
635 fails


In [134]:
len(img_list)

5255